# PyTorch の学習と推論を SageMaker のローカルモードで行う

#### ノートブックに含まれる内容

- PyTorch の学習と推論を SageMaker のローカルモードで行うやりかた

#### ノートブックで使われている手法の詳細

- アルゴリズム: MNIST
- データ: CNN

まず，ローカル実行に必要なコンポーネントをインストールします．セットアップ手順はシェルスクリプトにまとまっているので，これを実行します．

In [ ]:
!/bin/bash ./setup.sh

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()

role = sagemaker.get_execution_role()

## データのロード

ここでは，`PyTorch` でサポートされている関数を使って，MNIST データをダウンロードします．SageMaker の学習時につかうデータは，S3 に置く必要があります．ここでは，ローカルに落とした MNIST データを npz 形式で固めてから，SageMaker のラッパー関数を使って S3 にアップロードします．

デフォルトでは SageMaker は `sagemaker-{region}-{your aws account number}` というバケットを使用します．当該バケットがない場合には，自動で新しく作成します．`upload_data()` メソッドの引数に bucket=XXXX という形でデータを配置するバケットを指定することが可能です

In [ ]:
from torchvision import datasets, transforms

datasets.MNIST('data', download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]))

以下を実行する前に，**<span style="color: red;">`sagemaker/DEMO-pytorch-mnist-XX` の XX を指定された適切な数字に変更</span>**してください．

In [ ]:
bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-mnist-XX'

inputs = sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

## PyTorch を SageMaker で使う際のポイント

学習については，main 関数内にそのままモデルを記述するだけで大丈夫です．あとは SageMaker 側で学習ジョブを走らせる際に，main 関数内の学習処理が実行されます．ですので，既存の PyTorch の実行スクリプトをほぼそのまま SageMaker に移植することができます．また，環境変数経由で入力データの場所や GPU の数などを取得することが可能です．これは `argparse` 経由で `main` 関数内で受け取ることができます．詳細は[こちら](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/pytorch/README.rst)をご覧ください．

また推論時の処理は，`model_fn` で学習済みモデルをロードする部分だけ記述する必要があります．その他オプションで，前処理，推論処理，後処理部分を `input_fn`, `predict_fn`, `output_fn` で書くこともできます．デフォルトでは，`application/x-npy` コンテントタイプで指定される，NumPy 配列を入力として受け取ります． 


In [ ]:
!pygmentize mnist.py

## モデルの学習を実行

以下のように `PyTorch` オブジェクトを作成し，`fit()` メソッドで学習ジョブを実行します． `entry_point` で指定したローカルのスクリプトが，学習用のコンテナ内で実行されます．また合わせてローカルの `source_dir` を指定することで，依存するスクリプト群をコンテナにコピーして，学習時に使用することが可能です．そしてここでは，`train_instance_type` に `local` と指定しています．これにより，学習用コンテナをノートブックインスタンスのローカルに落としてきて，学習を行います．

In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="mnist.py",
                    role=role,
                    framework_version='0.4.0',
                    train_instance_count=1,
                    train_instance_type='local',
                    hyperparameters={
                        'epochs': 3,
                        'backend': 'gloo'
                    })

estimator.fit({'training': inputs})

# モデルの推論を実行


推論を行うために，まず学習したモデルをデプロイします．`deploy()` メソッドでは，デプロイ先エンドポイントのインスタンス数，インスタンスタイプを指定します．こちらも同様に `local` としておくことで，ローカルにエンドポイントがすぐ立ち上がります．

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='local')

あとは，実際にキャンバスに文字を描いて，予測を行なってみましょう．

In [ ]:
from IPython.display import HTML
HTML(open("input.html").read())

In [ ]:
import numpy as np

image = np.array([data], dtype=np.float32)
response = predictor.predict(image)
prediction = response.argmax(axis=1)[0]
print(prediction)

## エンドポイントの削除

全て終わったら，エンドポイントを削除します．

In [ ]:
estimator.delete_endpoint()